In [ ]:
# multivariate multi-step stacked lstm example
import numpy as np
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import pandas as pd
import pickle as pkl

In [ ]:
# RLDD
rldd_scaled = pd.read_pickle('../input/rlddandmerged/rldd_normalized_scaled.pkl')

# get video info from other dataset
rldd_UNscaled = pd.read_pickle('../input/rlddandmerged/rldd_normalized.pkl')
video_info_df2 = rldd_UNscaled.iloc[:,0:4]

# and concat video info with scaled df
rldd = pd.concat([video_info_df2, rldd_scaled], axis=1)

# fix drowsiness column
rldd["DROWSINESS"] = rldd.loc[:, "DROWSINESS"].map({0:0, 0.5:1, 1:2})

In [ ]:
# add rldd videos to lists
g = rldd.groupby(["fold", "subject", "videoname"])
len(g.groups.keys())

rldd_train_list = []
rldd_test_list = []

for key in g.groups.keys():
    print(key)
    if key[1] in ['57', '58', '59', '60']:
        rldd_test_list.append(g.get_group(key).iloc[:,4:])
    else:
        rldd_train_list.append(g.get_group(key).iloc[:,4:])

In [ ]:
# split a multivariate sequence into samples
def split_sequences(dataset_arr, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(dataset_arr)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the dataset
        if out_end_ix > len(dataset_arr) or end_ix > len(dataset_arr):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = dataset_arr[i:end_ix, :-1], dataset_arr[end_ix:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [ ]:
# split sequences for rldd train
rldd_train_seq_list_X=[]
rldd_train_seq_list_y=[]

n_steps_in, n_steps_out = 150, 60

for video_df in rldd_train_list:
    video_arr=np.array(video_df)
    # covert into input/output
    X, y = split_sequences(video_arr, n_steps_in, n_steps_out)
    if len(X)==0 or len(y)==0:
        continue
    rldd_train_seq_list_X.append(X)
    rldd_train_seq_list_y.append(y)

# concat lists
rldd_train_X = np.concatenate(rldd_train_seq_list_X, axis=0)
rldd_train_y = np.concatenate(rldd_train_seq_list_y, axis=0)

pkl.dump(rldd_train_X, open("rldd_train_X", 'wb'))
pkl.dump(rldd_train_y, open("rldd_train_y", 'wb'))

In [ ]:
# split sequences for rldd test
rldd_test_seq_list_X=[]
rldd_test_seq_list_y=[]

n_steps_in, n_steps_out = 150, 60

for video_df in rldd_test_list:
    video_arr=np.array(video_df)
    # covert into input/output
    X, y = split_sequences(video_arr, n_steps_in, n_steps_out)
    if len(X)==0 or len(y)==0:
        continue
    rldd_test_seq_list_X.append(X)
    rldd_test_seq_list_y.append(y)

# concat lists
rldd_test_X = np.concatenate(rldd_test_seq_list_X, axis=0)
rldd_test_y = np.concatenate(rldd_test_seq_list_y, axis=0)

pkl.dump(rldd_test_X, open("rldd_test_X", 'wb'))
pkl.dump(rldd_test_y, open("rldd_test_y", 'wb'))